<a href="https://colab.research.google.com/github/LuizBattaglia/DoubleTea/blob/master/produtos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introdução

O objetivo dessa seção é o de apresentar a relação entre alguns instrumentos financeiros disponíveis no mercado, tendo um ponto de vista macroeconômico.^[Essa seção está baseada em @assafneto]

# Taxa básica de juros

A taxa básica de juros é, conforme já discutimos, definida pela taxa Selic, servindo como referência para o mercado financeiro.

Como também vimos, a Selic é usada também para estabelecer o custo do dinheiro nas operações de mercado aberto com títulos públicos (o *open market* do Banco Central). A taxa média dessas operações em um dia é conhecida no mercado por **Taxa Média Selic (TMS)**.

Nesse contexto, a Selic é uma taxa *over*, válida para dias úteis, e definida para operações realizadas com títulos públicos de emissão do Governo Federal e registrados no Sistema Especial de Liquidação e Custódia (SELIC).

É a taxa de *mais baixo risco* no mercado financeiro brasileiro, sendo uma espécie de referência para as demais taxas de juros do mercado.

A metodologia de cálculo da taxa média diária adotada pelo Banco Central é a seguinte:^[Lembre-se da diferença entre a selic efetiva e a selic definida pelo COPOM.]

$$
Selic_{\text{Média}} = \left [ \left (\frac{\sum_{j=1}^{n} F_j + V_j}{\sum_{j=1}^{n}  V_j} \right )^{252} - 1 \right ] \times 100
$$

onde $F_j$ é um fator diário da taxa de juro de cada operação, $V_j$ é o valor de cada operação e $252$ é o número de dias úteis anuais definido pelo Banco Central.^[Dada uma taxa anual de 16%, a taxa equivalente por dia úteil será $Selic_{du} = (1 + 0.16)^{\frac{1}{252}} -1 = 0.0589\%$]

In [ ]:
from bcb import sgs
from plotnine import *
from functools import reduce
import pandas as pd

Podemos consultar as taxas no [SGS - Sistema Gerenciador de Séries Temporais](https://www3.bcb.gov.br/sgspub/localizarseries/localizarSeries.do?method=prepararTelaLocalizarSeries)
 - Mercados Financeiros e de capitais
  - Indicadores do mercado financeiro
   - Taxas de juros

In [ ]:
# Selic acumulada no mês anualizada base 252 (%a.a)
selic = sgs.get({'selic' : 4189}, start = '1999-01-01').reset_index()

(
  ggplot(selic, aes(x = 'Date', y = 'selic'))
 + geom_line(size = 1.2, color='#282f6b')
)

# Selic diária (%a.d)
selic_d = sgs.get({'selic_d' : 11}, start = '1999-01-01').reset_index()

# Selic anualizada base 252 diária (%a.a)
selic_an = sgs.get({'selic_an' : 1178}, start = '1999-01-01').reset_index()

# Selic acumulada no mês (%a.m)
selic_m = sgs.get({'selic_m' : 4390}, start = '1999-01-01').reset_index()

# Decomposição da Taxa Selic

A taxa Selic de um determinado período pode sser decomposta em duas partes: taxa real de juros e taxa de inflação:

$$
Selic = \left [(1 + i_{real}) \times (1 + INF)  \right ] - 1
$$

a taxa real por sua vez pode ser desmembrada em

$$
i_{real} = \left [(1 + i_{pura}) \times (1 + i_{risco})  \right ] - 1
$$

onde $i_{pura}$ é a taxa livre de risco e $i_{risco}$ é o risco mínimo da economia.

Assim, a taxa Selic pode ser apresentada como

$$
Selic = \left [(1 + i_{pura}) \times (1 + i_{risco}) \times (1 + INF)  \right ] - 1
$$

Por exemplo, sendo a Selic fixada em 15.5% a.a., a inflação prevista estando em 4.8% e supondo uma taxa livre de risco de 6%, pode-se calcular a taxa de risco da economia como:

$$
i_{risco} = \frac{1.155}{(1.06) \times (1.048)} - 1 = 3.97 \nonumber
$$

# Certificado de Depósito Bancário (CDB)

Os CDBs são títulos de renda fixa, representativos de depósitos a prazo realizados por pessoas físicas e jurídicas, e emitidos por Bancos Comerciais/Múltiplos.

Os recursos gerados pela colocação desses títulos são destinados principalmente a lastrear operações de financiamento de capital de giro. A subscrição de um CDB constitui-se, nesse sentido, numa forma de empréstimo que um cliente efetua para o banco, tendo como remuneração os juros pagos pela instituição financeira.

A remuneração de um CDB pode ser via taxa *prefixada*, *pós-fixada* ou *flutuante*.

Podemos consultar as taxas no [SGS - Sistema Gerenciador de Séries Temporais](https://www3.bcb.gov.br/sgspub/localizarseries/localizarSeries.do?method=prepararTelaLocalizarSeries)
 - Mercados Financeiros e de capitais
  - Depósitos a prazo

In [ ]:
# (CDB/RDB) - Taxa média mensal prefixada - Total (% a.a.)
cdb_pre = sgs.get({'cdb_pre' : 3954}, start = '1999-01-01').reset_index()

# (CDB/RDB) - Taxa média mensal pós-fixada - Total (% a.a.)
cdb_pos = sgs.get({'cdb_pos' : 28663}, start = '1999-01-01').reset_index()

# (CDB/RDB) - Taxa média flutuante DI total (%a.a) - diária!
cdb_f = sgs.get({'cdb_f' : 1157}, start = '1999-01-01').reset_index()

# lista de dfs para juntar
cdbs = [cdb_pre, cdb_pos, selic]

# juntar dados
cdbs_df = reduce(lambda left, right: pd.merge(left, right, on = 'Date'), cdbs)

# transforma de wide para long
cdbs_df_long = cdbs_df.melt(id_vars = ['Date'], value_name = 'taxas')

# cria o gráfico
(
 ggplot(cdbs_df_long, aes(x = 'Date', y = 'taxas', color = 'variable'))
 + geom_line(size = 0.7)
 + theme_minimal()
)

# Certificado de Depósito Interfinanceiro (CDI)

Podemos consultar as taxas no [SGS - Sistema Gerenciador de Séries Temporais](https://www3.bcb.gov.br/sgspub/localizarseries/localizarSeries.do?method=prepararTelaLocalizarSeries)
 - Mercados Financeiros e de capitais
  - Indicadores do mercado financeiro
   - Taxas de juros

In [ ]:
#	CDI acumulada no mês anualizada base 252
cdi = sgs.get({'cdi' : 4392}, start = '1999-01-01').reset_index()

# CDI diária (%a.d)
cdi_d = sgs.get({'cdi_d' : 12}, start = '1999-01-01').reset_index()

# CDI anualizada base 252 diária (%a.a)
cdi_an = sgs.get({'cdi_an' : 4389}, start = '1999-01-01').reset_index()

# Selic acumulada no mês (%a.m)
cdi_m = sgs.get({'cdi_m' : 4391}, start = '1999-01-01').reset_index()

In [ ]:
dfs = pd.merge(cdi, cdbs_df, on = 'Date').melt(id_vars = ['Date'], value_name = 'taxas')

# cria o gráfico
(
 ggplot(dfs, aes(x = 'Date', y = 'taxas', color = 'variable'))
 + geom_line(size = 0.7)
 + theme_minimal()
)

# TJLP e TLP

A TJLP foi uma taxa de juros aplicada preferencialmente em operações de longo prazo, constituída de maneira a viabilizar as operações de maior maturidade. Foi abandonada em 2018, sendo substituída pela TLP.

Para cálculo da TJLP eram consideradas a taxa meta de inflação calculada para o seu período de vigência e as taxas de juros dos títulos da dívida interna e externa do Brasil, admitidas quando de suas respectivas emissões no mercado primário.

Podemos obter a sua série, de forma a poder analisar seu valores passados e vigentes (para contratos ainda vigentes).

In [ ]:
# Taxa de juros de longo prazo - TJLP (%a.a)
tjlp = sgs.get({'tjlp' : 256}, start = '1999-01-01').reset_index()

# cria o gráfico
(
 ggplot(tjlp, aes(x = 'Date', y = 'tjlp'))
 + geom_line(size = 0.7)
 + theme_minimal()
)

Para o caso da TLP, obtemos a parcela de juros usando o código abaixo.

In [ ]:
# Jm - Taxa mensal para cálculo da TLP (%a.a)
tlp = sgs.get({'tlp' : 27572}, start = '1999-01-01').reset_index()

# cria o gráfico
(
 ggplot(tlp, aes(x = 'Date', y = 'tlp'))
 + geom_line(size = 0.7)
 + theme_minimal()
)